In [ ]:
# spell_checker
# !pip install git+https://github.com/ssut/py-hanspell.git

# !pip install konlpy

In [ ]:
#lib

# import re

from hanspell import spell_checker
from konlpy.tag import Kkma

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

In [ ]:
# token_ls path
token_ls_path = '/content/drive/MyDrive/Programming/Project/Hel_ri_celus (AI voice_bot for delivery_riders)/token_ls.csv'

# model_path
model_path = '/content/drive/MyDrive/Programming/Project/Hel_ri_celus (AI voice_bot for delivery_riders)/model/best_model.h5'

In [ ]:
# # 정규표현식 함수 정의

# def re_sub(self):
    
#     self = re.sub("\d\d\d\d"," @",self)
#     self = re.sub("\d\d분"," #분",self)

#     return self

In [ ]:
## 띄어쓰기, 맞춤법
def check_spell(self):
    
    spelled_sent = spell_checker.check(self)
    self = spelled_sent.checked

    return self

In [ ]:
# kkma 형태소 토큰화 함수정의
def kkma_tokenizer(self):
    kkma = Kkma()
    valid_pos = ['NNG','VV','SW','MAG']

    # tokenize
    token_text = kkma.pos(self)
    
    # 불용어 제거
    ls = []
    for token in token_text:
        
        if token[1] in valid_pos:
            ls.append(token[0])
        
    self = ls

    return self

In [ ]:
## df의 정수인코딩 column 생성 및 반영 함수화
# tokenizer.fit_on_texts 에서 특수문자 @,# 을 자체적으로 없앰, 이 부분 보완 필요

def int_encode(self):
    
    # 토큰 리스트 경로
    token_ls = pd.read_csv(token_ls_path)['token']
    
    # 토크나이저 최적화
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(token_ls)

    seq = tokenizer.texts_to_sequences(self)

    ls = sum(seq,[])

    self = ls

    return self

In [ ]:
def padding(self):
    max_len = 8

    pad = [0]*(max_len-len(self))
    self = pad + self

    return self

In [ ]:
def pred(self):

    self = np.array(self).reshape(-1,8)
    
    model = load_model(model_path)
    
    pred = model.predict(self)
    
    self = np.argmax(pred)

    
    return self

In [ ]:
def start_predict(self):

    # re_self = re_sub(self)
    check_self = check_spell(re_self)
    kkma_self = kkma_tokenizer(check_self)
    int_self = int_encode(kkma_self)
    pad_self = padding(int_self)
    pred_self = pred(pad_self)

    return pred_self

In [ ]:
text = "주문번호 3400"
start_predict(text)
